In [1]:
!pip install scanpy scperturb --quiet

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as pl
import seaborn as sns
import os
import sys
from tqdm.auto import tqdm
from pathlib import Path

In [3]:
# load config
import yaml
with open('../../configuration/config.yaml', 'r') as file:
    config = yaml.safe_load(file)
DOWNDIR = Path(config['DOWNDIR'])
TEMPDIR = Path(config['TEMPDIR'])

In [4]:
df = pd.read_csv(DOWNDIR / 'all_obs.csv', index_col=0)

/tmp/ipykernel_1131/4116760915.py:1: DtypeWarning: Columns (1,2,5,6,8,10,11,18,19,20,21,22,23,24,25,26,28,29,30,31,32,33,34,35,36,38,44,45,46,51,52,53,54,55,56,58,59,60,61,62,63,67,69,70,71,72,76,77,79,87,89,90,91,92,93,94,98,99,101,106,107,108,109,110,111,113,114,115,116,118,120,121,122,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(DOWNDIR / 'all_obs.csv', index_col=0)


In [24]:
df = df[df.ncounts > 500]  # basic filtering

In [25]:
M=df.groupby('dataset').median(numeric_only=True)[['ncounts', 'ngenes']].astype(int)

In [26]:
M

,ncounts,ngenes
dataset,,
AdamsonWeissman2016_GSM2406675_10X001,9969,2580
AdamsonWeissman2016_GSM2406677_10X005,25082,4304
AdamsonWeissman2016_GSM2406681_10X010,15355,3690
AissaBenevolenskaya2021,1312,764
ChangYe2021,18961,5278
DatlingerBock2017,6711,2713
DatlingerBock2021,1282,364
DixitRegev2016,13974,3080
FrangiehIzar2021_RNA,10988,3314


In [10]:
M

,ncounts,ngenes
dataset,,
AdamsonWeissman2016_GSM2406675_10X001,9969,2580
AdamsonWeissman2016_GSM2406677_10X005,25082,4304
AdamsonWeissman2016_GSM2406681_10X010,15355,3690
AissaBenevolenskaya2021,75,49
ChangYe2021,18961,5278
DatlingerBock2017,6711,2713
DatlingerBock2021,460,179
DixitRegev2016,13974,3080
FrangiehIzar2021_RNA,10988,3314


In [20]:
sdf = df[df.dataset=='DatlingerBock2021']

In [21]:
sdf

,cell_barcode,perturbation,read count,UMI count,tissue_type,cell_line,cancer,disease,perturbation_type,celltype,...,well_id,sgRNA_read_count,sgRNA_umi_count,percent_hemo,library_names,filename,Sample,tissue,location,diagnosis
0,A01-ACTTTCAGTAACGGAC,ZAP70_1,NaN,NaN,cell_line,Jurkat cells,True,acute T cell leukemia,CRISPR,T cells,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A01-AGCTGATTCGCTGATA,ZAP70_1,NaN,NaN,cell_line,Jurkat cells,True,acute T cell leukemia,CRISPR,T cells,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A01-CCTTAATTCCAGAGAG,ZAP70_1,NaN,NaN,cell_line,Jurkat cells,True,acute T cell leukemia,CRISPR,T cells,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A01-CTCTAGCGTCTGGGAA,ZAP70_1,NaN,NaN,cell_line,Jurkat cells,True,acute T cell leukemia,CRISPR,T cells,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A01-GCGATTAGTGATAGAT,ZAP70_1,NaN,NaN,cell_line,Jurkat cells,True,acute T cell leukemia,CRISPR,T cells,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39189,P24-TTCATTGTCTCCACTG,control,NaN,NaN,cell_line,Jurkat cells,True,acute T cell leukemia,CRISPR,T cells,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39190,P24-TTCGATTCATAAAGTG,control,NaN,NaN,cell_line,Jurkat cells,True,acute T cell leukemia,CRISPR,T cells,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39191,P24-TTGCACCGTCCGGTAC,control,NaN,NaN,cell_line,Jurkat cells,True,acute T cell leukemia,CRISPR,T cells,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39192,P24-TTGTCTATCAAACCAC,control,NaN,NaN,cell_line,Jurkat cells,True,acute T cell leukemia,CRISPR,T cells,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
sdf = sdf[sdf.ncounts > 500]

In [23]:
sdf.ncounts.median(), sdf.ngenes.median()

(1282.0, 364.0)